# Тема: " Система управления базами данных MongoDB в Python"

### Задание 1

Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

#### Ответ:

In [1]:
# Импорт необходимых библиотек:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
from time import sleep

#### Парсинг вакансий с сайта HeadHanter

In [2]:
# Обозначение URL сайта HeadHanter:
URL_HH = 'https://hh.ru/search/vacancy'

In [3]:
# Устновка User-Agent:
HEADER = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
          AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

In [4]:
# Функция запроса HTML-страницы по URL сайта:
def get_html(url, params=''):
    response = requests.get(url, headers = HEADER, params = params)
    
    return response

#response = get_html(URL_HH, params={'text': 'Разработчик Python', 'page': 0})

In [5]:
# Функция запроса контента с HTML-страницы HeadHanter:
def get_hh_content(response):
    soup = bs(response.text, 'html.parser')
    items = soup.find_all('div', {'class': 'serp-item'})
    
    vacancy = []
    for item in items:
        title = item.find('a', {'class': 'serp-item__title'}).get_text()
        link = item.find('a', {'class': 'serp-item__title'})['href']
        city = item.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).contents[0].strip(', ')
        company = item.find('a', {'class': 'bloko-link_kind-tertiary'}).get_text()
        try:
            salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).get_text()
        except:
            salary = ''

        vacancy.append(
            {'title': title,
             'link': link,
             'city': city,
             'company': company,
             'salary': salary,
             'site': 'HeadHunter'})
        
    return vacancy

# get_hh_content(response)

In [6]:
# Функция упорядочения полученных данных и добавления Id вакансии:
def edit_data_hh(vacancy):
    for vac in vacancy:
        # Del \xa0
        vac['company'] = vac['company'].replace(u'\xa0', u' ')
        vac['salary'] = vac['salary'].replace(u'\u202f', u'')

        # Salary (min, max, currency)
        if vac['salary']:
            salary_list = vac['salary'].split(' ')
            if salary_list[0] == 'от':
                vac['salary_min'] = float(salary_list[1])
                vac['salary_max'] = None
            elif salary_list[0] == 'до':
                vac['salary_min'] = None
                vac['salary_max'] = float(salary_list[1])
            else:
                vac['salary_min'] = float(salary_list[0])
                vac['salary_max'] = float(salary_list[2])
            vac['salary_currency'] = salary_list[-1]
        else:
            vac['salary_min'] = None
            vac['salary_max'] = None
            vac['salary_currency'] = None
        vac.pop('salary')

        # Id
        id = vac['link'].split(sep='/')[4].split(sep='?')[0]
        vac['_Id'] = id
    
    return vacancy

In [7]:
# Главная функция парсинга сайта HeadHanter:
def parser_hh():
    post = str(input('Введите название вакансий для парсинга: '))
    pages = int(input('Введите количество страниц для парсинга: '))
    
    result = []
    for i in range(pages):
        print(f'Парсится hh, страница: {i + 1}')
        response = get_html(URL_HH, params = {'text': post, 'page': i})
        vacancy = get_hh_content(response)
        vacancy_edit = edit_data_hh(vacancy)
        if response.ok and vacancy:
            result.extend(vacancy_edit)
        else:
            break

    return result

result = parser_hh()
pprint(result)

Введите название вакансий для парсинга: Разработчик Python
Введите количество страниц для парсинга: 1
Парсится hh, страница: 1
[{'_Id': '77147428',
  'city': 'Москва',
  'company': 'MANGO FZCO',
  'link': 'https://vladivostok.hh.ru/vacancy/77147428?from=vacancy_search_list&query=%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA+Python',
  'salary_currency': 'руб.',
  'salary_max': None,
  'salary_min': 120000.0,
  'site': 'HeadHunter',
  'title': 'Backend программист (Python)'},
 {'_Id': '73199414',
  'city': 'Санкт-Петербург',
  'company': 'ЦРТ | Группа компаний',
  'link': 'https://vladivostok.hh.ru/vacancy/73199414?from=vacancy_search_list&query=%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA+Python',
  'salary_currency': None,
  'salary_max': None,
  'salary_min': None,
  'site': 'HeadHunter',
  'title': 'Ведущий инженер по тестированию (python)'},
 {'_Id': '76968960',
  'city': 'Москва',
  'company': 'СБЕРКОРУС',
  'link': 'https://vladivostok.h

#### Развёртывание MongoDB

In [8]:
# Установка драйвера PyMongo в среду Python, используя Сonda:
#!conda install -c anaconda pymongo

In [9]:
# Использование объекта MongoClien для подключуния к MongoDB с помощью PyMongo:
import pymongo
from pymongo import MongoClient
import pymongo.errors

In [10]:
# Создание экземпляра клиента для mongod:
client = MongoClient('mongodb://localhost:27017/')
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [11]:
# Создание базы данных (БД):
db = client.headhunter
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'headhunter')

In [12]:
# Создание коллекции в БД:
vacancies = db.vacancies
vacancies

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'headhunter'), 'vacancies')

In [13]:
# Функция добавления только новых вакансий в коллекцию vacancies:
def insert_mongo(result):
    for vac in result:
        try:
            vacancies.update_one({'_Id': vac['_Id']}, {'$set': vac}, upsert = True)
        except idoc:
            print(f'Error {idoc}')
            continue

insert_mongo(result)

In [14]:
# Просмотр документов в коллекции vacancies:
cursor = vacancies.find()

for record in cursor:
    pprint(record)

{'_Id': '77147428',
 '_id': ObjectId('63f759e05e936cb6b7a669f6'),
 'city': 'Москва',
 'company': 'MANGO FZCO',
 'link': 'https://vladivostok.hh.ru/vacancy/77147428?from=vacancy_search_list&query=%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA+Python',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': 120000.0,
 'site': 'HeadHunter',
 'title': 'Backend программист (Python)'}
{'_Id': '73199414',
 '_id': ObjectId('63f759e05e936cb6b7a669f8'),
 'city': 'Санкт-Петербург',
 'company': 'ЦРТ | Группа компаний',
 'link': 'https://vladivostok.hh.ru/vacancy/73199414?from=vacancy_search_list&query=%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA+Python',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'HeadHunter',
 'title': 'Ведущий инженер по тестированию (python)'}
{'_Id': '76968960',
 '_id': ObjectId('63f759e05e936cb6b7a669fa'),
 'city': 'Москва',
 'company': 'СБЕРКОРУС',
 'link': 'https://vladivostok.hh.ru/vacancy/

In [15]:
# Подсчёт количества документов в коллекции vacancies:
cursor = len(list(vacancies.find()))
print('Количество документов в коллекции:', cursor)

Количество документов в коллекции: 20


### Задание 2

Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля).

#### Ответ:

In [16]:
# Функция поиска и вывода на экран вакансий с ЗП больше введённой суммы:
def search_by_salary():
    value = int(input('Введите пороговое значение заработной платы: '))   
    res = vacancies.find({'$or': [{'salary_currency': 'руб.',
                                   '$or': [{'salary_min': {'$gt': value}},
                                           {'salary_max': {'$gt': value}},
                                          ]},
                                  {'salary_currency': 'USD',
                                   '$or': [{'salary_min': {'$gt': value / 75}},
                                           {'salary_max': {'$gt': value / 75}},
                                          ]
                                 }]
                        })
    
    return (list(res))

res = search_by_salary()
pprint(res)

Введите пороговое значение заработной платы: 250000
[{'_Id': '72216524',
  '_id': ObjectId('63f759e05e936cb6b7a66a06'),
  'city': 'Москва',
  'company': 'ООО Перфект Системс',
  'link': 'https://vladivostok.hh.ru/vacancy/72216524?from=vacancy_search_list&query=%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA+Python',
  'salary_currency': 'руб.',
  'salary_max': 450000.0,
  'salary_min': 300000.0,
  'site': 'HeadHunter',
  'title': 'Python Backend Developer / WEB - разработчик'},
 {'_Id': '76832091',
  '_id': ObjectId('63f759e05e936cb6b7a66a16'),
  'city': 'Москва',
  'company': 'Apostro',
  'link': 'https://vladivostok.hh.ru/vacancy/76832091?from=vacancy_search_list&query=%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA+Python',
  'salary_currency': 'USD',
  'salary_max': 8000.0,
  'salary_min': None,
  'site': 'HeadHunter',
  'title': 'Ведущий Python разработчик (удаленка)'},
 {'_Id': '71147348',
  '_id': ObjectId('63f759e05e936cb6b7a66a1a'),
  'cit

In [17]:
print('Количество найденных вакансий, удовлетворяющих условию задания:', len(list(res)))

Количество найденных вакансий, удовлетворяющих условию задания: 3


In [18]:
# Очистка коллекции:
#result = vacancies.delete_many({})
#print(result.deleted_count)

---

END